In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import pandas as pd
from DateProvider import DateProvider
from datetime import datetime
from ColumnsAdder import ColumnsAdder
from MedianOfFreeBedsByKreisTableFactory import MedianOfFreeBedsByKreisTableFactory
from TimeseriesReader import readTimeseries
from IntensivstationenHtmlFileUpdater import saveLastUpdatedIntensivstationen, saveKreisOptions
from KreiseReader import readKreise
from KreisOptionsProvider import getKreisOptionsAndAlleLandkreise
from IntensiveCareBedsPersister import getAndPersistIntensiveCareBeds4AlleKreise
from Chart import createMedianOfFreeBedsByKreisChart


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', 'raise')


In [3]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

17.01.2023, 15:45:59 Uhr


In [4]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

          lastUpdated: 2023-01-17 11:49:00
lastUpdatedDataSource: 2023-01-17 11:49:00
needsUpdate: False


In [5]:
saveLastUpdatedIntensivstationen(
    dateProvider.getLastUpdatedDataSource(),
    toHtmlFile = "../../docs/intensivstationen.html")

In [6]:
timeSeries = readTimeseries(download = needsUpdate)
timeSeries

FileNotFoundError: [Errno 2] No such file or directory: 'zeitreihe-tagesdaten.csv'

In [7]:
kreisValues = sorted(timeSeries['Kreis'].drop_duplicates().values)
saveKreisOptions(
    kreisOptions = getKreisOptionsAndAlleLandkreise(kreisValues),
    toHtmlFile = "../../docs/intensivstationen.html")

NameError: name 'timeSeries' is not defined

In [8]:
getAndPersistIntensiveCareBeds4AlleKreise(
    timeSeries,
    intensivstationenDataDir = '../../docs/data/intensivstationen',
    kreisValues = kreisValues)


NameError: name 'timeSeries' is not defined

In [9]:
medianOfFreeBedsByKreisTable = MedianOfFreeBedsByKreisTableFactory(timeSeries).createMedianOfFreeBedsByKreisTable('Kreis')
medianOfFreeBedsByKreisTable.reset_index().to_json(
    '../../docs/data/intensivstationen/medianOfFreeBedsByKreisTable.json',
    orient = "records")

NameError: name 'timeSeries' is not defined

In [10]:
def getChartTitle(dateStart, dateEnd):
    def formatDate(date):
        return date.strftime("%d.%m.%Y")

    return f"Median freier Intensivbetten im Zeitraum {formatDate(dateStart)} bis {formatDate(dateEnd)}"

In [11]:
createMedianOfFreeBedsByKreisChart(
    timeSeries,
    chartTitle = getChartTitle(
        dateStart=timeSeries['date'].min(),
        dateEnd=timeSeries['date'].max()))

NameError: name 'timeSeries' is not defined